In [35]:
'''
https://maps.mail.ru/osm/tools/overpass/#

[out:json];
area[name="Белгородская область"];
node(area)["place"]->.nodes;
.nodes is_in ->.trees;
area.trees["boundary"="administrative"]["admin_level"~"8"]->.area;
rel(pivot.area.area);
out geom;
'''



import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

reg=gpd.read_file('./BEL.geojson')
reg=reg[reg['admin_level']=='8'][['name','geometry']]


rdb=pd.read_excel('./RDB.xlsx')
rdb=rdb[rdb['MNEMO']=='BEL'][['SiteID','Lon','Lat']]

population=pd.read_csv('population.csv')
population=population[population['region']=='Белгородская область'][['settlement','population','latitude_dd','longitude_dd']].reset_index(drop=True).rename(columns={'latitude_dd':'Lat','longitude_dd':'Lon'})


def retRegID(rdb):
    return (list(filter(lambda x:x[1],enumerate(reg.contains(Point(rdb['Lon'],rdb['Lat']))))) or [(None,None)])[0][0]

population['reg_id']=population.apply(lambda x:retRegID(x),axis=1)
population=population.dropna()
population=population.join(reg,on='reg_id').drop(['geometry'],axis=1)[['name','reg_id','population']].groupby(['name','reg_id']).sum()

rdb['reg_id']=rdb.apply(lambda x:retRegID(x),axis=1)
rdb=rdb.dropna()
rdb=rdb.join(reg,on='reg_id').drop(['geometry'],axis=1).join(pd.read_excel('./BSM.xlsx').set_index('SiteID'),on='SiteID')
rdb=rdb[['name','reg_id','Period','Subs']].groupby(['name','reg_id','Period'],as_index=False).sum()
rdb=rdb.join(population,on=['name','reg_id'])
rdb.to_excel('golf.xlsx')



In [126]:
#2

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

def readGEO(file,admin_level):
    geo=gpd.read_file(file)
    return geo[geo['admin_level']==str(admin_level)][['name','geometry']]

def retParentID(parent,geom):
    loc=parent[parent.contains(geom)]
    return loc.index[0] if len(loc)>0 else None


#Читаем гео-данные в формате ID, name, geometry
reg,distr,rurals=readGEO('./BEL/reg.geojson',4),readGEO('./BEL/distr.geojson',6),readGEO('./BEL/county.geojson',8)

#Читаем справочник н.п. в формате id, settlement, population, Lat, Lon
population=pd.read_csv('population.csv')
population=population[population['region']=='Белгородская область'][['settlement','population','latitude_dd','longitude_dd']].reset_index(drop=True).rename(columns={'latitude_dd':'Lat','longitude_dd':'Lon'})

#1. Привязываем population к rural_ID и к distr_ID (т.к. не все села относятся к сельским поселениям)
population['rural_ID'],population['distr_ID']=population.apply(lambda x:retParentID(rurals,Point(x['Lon'],x['Lat'])),axis=1),population.apply(lambda x:retParentID(distr,Point(x['Lon'],x['Lat'])),axis=1)

#2. Привязываем rural_ID к distr_ID
rurals['distr_ID']=rurals.apply(lambda x:retParentID(distr,x['geometry']),axis=1)

#3. Привязываем distr_ID к reg_ID
distr['reg_ID']=0

#4. JOIN

ds=population.join(rurals,on='rural_ID',rsuffix='_rural').join(distr,on='distr_ID',rsuffix='_distr').join(reg,on='reg_ID',rsuffix='_reg')

#5. Привязываем BS к rural

rdb=pd.read_excel('./RDB.xlsx')
rdb=rdb[rdb['MNEMO']=='BEL'][['SiteID','Lon','Lat']]
rdb['rural_ID']=rdb.apply(lambda x:retParentID(rurals,Point(x['Lon'],x['Lat'])),axis=1)

#5.5 Ищем ближайший н.п. к BS
pop,_rdb=gpd.GeoDataFrame(),gpd.GeoDataFrame()
pop['geometry']=population.apply(lambda x:Point(x['Lon'],x['Lat']),axis=1)
_rdb[['geometry','SiteID']]=pd.DataFrame(data={'col1':rdb.apply(lambda x:Point(x['Lon'],x['Lat']),axis=1),'col2':rdb['SiteID']})
pop=pop.sjoin_nearest(_rdb,distance_col='distance')
pop['distance']=pop['distance']*111
pop=pop[['SiteID','distance']]
pop=pop.groupby([pop.index]).agg({'SiteID':lambda x:','.join(list(x)),'distance':['mean']})
pop.columns=pop.columns.droplevel(1)
ds=ds.join(pop)

#6 Привязываем инфу по BS к BS
rdb=rdb.join(pd.read_excel('./BSM.xlsx').set_index('SiteID'),on='SiteID')[['Subs','Period','rural_ID']].dropna().groupby(['rural_ID','Period'],as_index=False).sum().set_index('rural_ID')

#rdb=rdb.pivot(columns='Period', values='Subs')

ds=ds[['name_reg','name_distr','name','settlement','population','SiteID','distance','rural_ID']]
ds=ds.join(rdb,on='rural_ID',rsuffix='_rdb')
#ds=ds.drop(['Lat','Lon','rural_ID','distr_ID','geometry','distr_ID_rural','geometry_distr','reg_ID','geometry_reg'])
ds=ds.drop(['rural_ID'],axis=1)
#ds.to_excel('_super_dic.xlsx',index=False)

In [1]:
#3

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

#1. Читаем данные из источников в формат GPD
print('#1. Читаем данные из источников в формат GPD')
loc=pd.read_csv('population.csv')
loc=loc[loc['region']=='Белгородская область'][['settlement','population','latitude_dd','longitude_dd']].reset_index(drop=True).rename(columns={'settlement':'location'})
loc['geometry']=loc.apply(lambda x:Point(x['longitude_dd'],x['latitude_dd']),axis=1)
loc=gpd.GeoDataFrame(loc[['location','population','geometry']]).rename(columns={'geometry':'geometry_loc'})

rdb=pd.read_excel('./RDB.xlsx')
rdb=rdb[rdb['MNEMO']=='BEL'][['SiteID','Lon','Lat']]
rdb['geometry']=rdb.apply(lambda x:Point(x['Lon'],x['Lat']),axis=1)
rdb=gpd.GeoDataFrame(rdb[['SiteID','geometry']])

ddb=pd.read_excel('./DDB.xlsx')
ddb=ddb[(ddb['MNEMO']=='BEL') & (ddb['PST_NAME']=='работает')][['POS_ID','Lon','Lat']]
ddb['geometry']=ddb.apply(lambda x:Point(x['Lon'],x['Lat']),axis=1)
ddb=gpd.GeoDataFrame(ddb[['POS_ID','geometry']])


#2 Формируем справочник по территориям
print('#2 Формируем справочник по территориям')
def readGEO(file,admin_level):
    geo=gpd.read_file(file)
    return geo[geo['admin_level']==str(admin_level)][['name','geometry']]

reg,distr,county=readGEO('./BEL/reg.geojson',4),readGEO('./BEL/distr.geojson',6),readGEO('./BEL/county.geojson',8)

#2.1 Привязываем loc к county и distr (т.к. некоторые населенники принадлежат непосредственно району)
print('#2.1 Привязываем loc к county и distr (т.к. некоторые населенники принадлежат непосредственно району)')
loc['county_ID']=loc.apply(lambda x:next(iter(county[county.contains(x['geometry_loc'])].index.values),None),axis=1)
loc['distr_ID']=loc.apply(lambda x:next(iter(distr[distr.contains(x['geometry_loc'])].index.values),None),axis=1)

#2.2 Привязываем county к distr
print('#2.2 Привязываем county к distr')
county['distr_ID']=county.apply(lambda x:next(iter(distr[distr.contains(x['geometry'])].index.values),None),axis=1)

#2.3. Привязываем distr_ID к reg_ID
print('#2.3. Привязываем distr_ID к reg_ID')
distr['reg_ID']=0

#2.4 Объединяем
print('#2.4 Объединяем')
ds=loc.join(county,on='county_ID',rsuffix='_county').join(distr,on='distr_ID',rsuffix='_distr').join(reg,on='reg_ID',rsuffix='_reg')
ds=ds.rename(columns={'name':'name_county','geometry':'geometry_county'})

#3 Ищем ближайшую BS к каждой loc
print('#3 Ищем ближайшую BS к каждой loc')
tmp=gpd.GeoDataFrame(ds[['geometry_loc']]).rename(columns={'geometry_loc':'geometry'}).sjoin_nearest(rdb,distance_col='distance').groupby(level=0).agg({'SiteID':lambda x:','.join(list(x)),'distance':['mean']})
tmp.columns=tmp.columns.droplevel(1)
tmp['distance']=tmp['distance']*111
ds=ds.join(tmp)

#4 Привязываем BS к county и дополняем развернутой инфой
print('#4 Привязываем BS к county и дополняем развернутой инфой')
rdb['county_ID']=rdb.apply(lambda x:next(iter(county[county.contains(x['geometry'])].index.values),None),axis=1)
rdb=rdb.join(pd.read_excel('./BSM.xlsx')[['SiteID','Period','Subs']].set_index('SiteID'),on='SiteID').dropna().groupby(['county_ID','Period'],as_index=False).agg({'Subs':['sum'],'SiteID':lambda x:','.join(list(x))}).set_index('county_ID')
rdb.columns=rdb.columns.droplevel(1)

#5 Привязываем POS к county и дополняем развернутой инфой
print('#5 Привязываем POS к county и дополняем развернутой инфой')
ddb['county_ID']=ddb.apply(lambda x:next(iter(county[county.contains(x['geometry'])].index.values),None),axis=1)

#5.1 Привязываем GI 3M
print('#5.1 Привязываем GI 3M')
gi=pd.read_excel('./BEL_Source.xlsx')[['Код точки продаж','GI','Дата']].rename(columns={'Код точки продаж':'POS_ID'})
gi=gi[gi['Дата']>'2022-05-01'].groupby(['POS_ID']).sum()

ddb=ddb.join(gi,on='POS_ID')
ddb=ddb.groupby(['county_ID'],as_index=False).agg({'POS_ID':lambda x:','.join(list(map(str,x))),'GI':'sum'}).set_index('county_ID')


#6 Делаем красиво
print('#6 Делаем красиво')
ds=ds[['name_reg','name_distr','name_county','location','population','SiteID','distance','county_ID']]
ds=ds.join(rdb,on='county_ID',rsuffix='_rdb').join(ddb,on='county_ID',rsuffix='_ddb')
ds=ds.drop(['county_ID'],axis=1)
ds=ds.rename(columns={'name_reg':'Регион','name_distr':'Район','name_county':'Округ/поселение','location':'Населенный пункт','population':'Численность населения','SiteID':'Ближайший сайт','distance':'Расстояние','SiteID_rdb':'Сайты на территории округа','Period':'Период','Subs':'Sim под покрытием','POS_ID':'ТТ на территории округа','GI':'GI 3M на территории округа'})
#7 Сохраняем
print('#7 Сохраняем')
ds.to_excel('dic_BEL.xlsx',index=False)
ds

C:\Users\mikhail.libman\AppData\Local\Continuum\anaconda3\lib\site-packages\geopandas\_compat.py:115: UserWarning: The Shapely GEOS version (3.7.0-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


#1. Читаем данные из источников в формат GPD
#2 Формируем справочник по территориям
#2.1 Привязываем loc к county и distr (т.к. некоторые населенники принадлежат непосредственно району)
#2.2 Привязываем county к distr
#2.3. Привязываем distr_ID к reg_ID
#2.4 Объединяем
#3 Ищем ближайшую BS к каждой loc
#4 Привязываем BS к county и дополняем развернутой инфой
#5 Привязываем POS к county и дополняем развернутой инфой
#5.1 Привязываем GI 3M
#6 Делаем красиво
#7 Сохраняем


,Регион,Район,Округ/поселение,Населенный пункт,Численность населения,Ближайший сайт,Расстояние,Период,Sim под покрытием,Сайты на территории округа,ТТ на территории округа,GI 3M на территории округа
0,Белгородская область,Борисовский район,Белянское сельское поселение,Беленькое,1463,BE0542,0.853935,2020-04-01,833.0,BE0542,"309365,1057688,1085299,1088031",13.0
0,Белгородская область,Борисовский район,Белянское сельское поселение,Беленькое,1463,BE0542,0.853935,2020-05-01,758.0,BE0542,"309365,1057688,1085299,1088031",13.0
0,Белгородская область,Борисовский район,Белянское сельское поселение,Беленькое,1463,BE0542,0.853935,2020-06-01,890.0,BE0542,"309365,1057688,1085299,1088031",13.0
0,Белгородская область,Борисовский район,Белянское сельское поселение,Беленькое,1463,BE0542,0.853935,2020-07-01,826.0,BE0542,"309365,1057688,1085299,1088031",13.0
0,Белгородская область,Борисовский район,Белянское сельское поселение,Беленькое,1463,BE0542,0.853935,2020-08-01,811.0,BE0542,"309365,1057688,1085299,1088031",13.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1597,Белгородская область,Вейделевский район,Викторопольское сельское поселение,Якименков,2,BE2224,8.299270,2022-06-01,480.0,BE1329,NaN,NaN
1597,Белгородская область,Вейделевский район,Викторопольское сельское поселение,Якименков,2,BE2224,8.299270,2022-07-01,465.0,BE1329,NaN,NaN
1598,Белгородская область,Новооскольский городской округ,NaN,Покрово-Михайловка,781,BE0114,2.530363,NaN,NaN,NaN,NaN,NaN
1599,Белгородская область,Старооскольский городской округ,NaN,Шаталовка,1339,BE0411,1.472048,NaN,NaN,NaN,NaN,NaN


In [3]:
rdb

,Period,Subs,SiteID
county_ID,,,
0.0,2020-04-01,671.0,BE0740
0.0,2020-05-01,692.0,BE0740
0.0,2020-06-01,730.0,BE0740
0.0,2020-07-01,750.0,BE0740
0.0,2020-08-01,744.0,BE0740
...,...,...,...
189.0,2022-03-01,191.0,BE2355
189.0,2022-04-01,208.0,BE2355
189.0,2022-05-01,217.0,BE2355


In [8]:
#4_VOL

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

#1. Читаем данные из источников в формат GPD
print('#1. Читаем данные из источников в формат GPD')
loc=pd.read_csv('population.csv')
loc=loc[loc['region']=='Вологодская область'][['settlement','population','latitude_dd','longitude_dd']].reset_index(drop=True).rename(columns={'settlement':'location'})
loc['geometry']=loc.apply(lambda x:Point(x['longitude_dd'],x['latitude_dd']),axis=1)
loc=gpd.GeoDataFrame(loc[['location','population','geometry']]).rename(columns={'geometry':'geometry_loc'})

rdb=pd.read_excel('./VOL/RDB.xlsx')
rdb=rdb[['SiteID','Lon','Lat']]
rdb['geometry']=rdb.apply(lambda x:Point(x['Lon'],x['Lat']),axis=1)
rdb=gpd.GeoDataFrame(rdb[['SiteID','geometry']])

#2 Формируем справочник по территориям
print('#2 Формируем справочник по территориям')
def readGEO(file,admin_level):
    geo=gpd.read_file(file)
    return geo[geo['admin_level']==str(admin_level)][['name','geometry']]

reg,distr,county=readGEO('./VOL/reg.geojson',4),readGEO('./VOL/distr.geojson',6),readGEO('./VOL/county.geojson',8)

#2.1 Привязываем loc к county и distr (т.к. некоторые населенники принадлежат непосредственно району)
print('#2.1 Привязываем loc к county и distr (т.к. некоторые населенники принадлежат непосредственно району)')
loc['county_ID']=loc.apply(lambda x:next(iter(county[county.contains(x['geometry_loc'])].index.values),None),axis=1)
loc['distr_ID']=loc.apply(lambda x:next(iter(distr[distr.contains(x['geometry_loc'])].index.values),None),axis=1)

#2.2 Привязываем county к distr
print('#2.2 Привязываем county к distr')
county['distr_ID']=county.apply(lambda x:next(iter(distr[distr.contains(x['geometry'])].index.values),None),axis=1)

#2.3. Привязываем distr_ID к reg_ID
print('#2.3. Привязываем distr_ID к reg_ID')
distr['reg_ID']=0

#2.4 Объединяем
print('#2.4 Объединяем')
ds=loc.join(county,on='county_ID',rsuffix='_county').join(distr,on='distr_ID',rsuffix='_distr').join(reg,on='reg_ID',rsuffix='_reg')
ds=ds.rename(columns={'name':'name_county','geometry':'geometry_county'})

#3 Ищем ближайшую BS к каждой loc
print('#3 Ищем ближайшую BS к каждой loc')
tmp=gpd.GeoDataFrame(ds[['geometry_loc']]).rename(columns={'geometry_loc':'geometry'}).sjoin_nearest(rdb,distance_col='distance').groupby(level=0).agg({'SiteID':lambda x:','.join(list(x)),'distance':['mean']})
tmp.columns=tmp.columns.droplevel(1)
tmp['distance']=tmp['distance']*111
ds=ds.join(tmp)

#4 Привязываем BS к county и дополняем развернутой инфой
print('#4 Привязываем BS к county и дополняем развернутой инфой')
rdb['county_ID']=rdb.apply(lambda x:next(iter(county[county.contains(x['geometry'])].index.values),None),axis=1)
rdb=rdb.join(pd.read_excel('./VOL/BSM_VOL.xlsx')[['SiteID','Period','Subs']].set_index('SiteID'),on='SiteID').dropna().groupby(['county_ID','Period'],as_index=False).agg({'Subs':['sum'],'SiteID':lambda x:','.join(list(x))}).set_index('county_ID')
rdb.columns=rdb.columns.droplevel(1)

#6 Делаем красиво
print('#6 Делаем красиво')
ds=ds[['name_reg','name_distr','name_county','location','population','SiteID','distance','county_ID']]
ds=ds.join(rdb,on='county_ID',rsuffix='_rdb')
ds=ds.drop(['county_ID'],axis=1)
ds=ds.rename(columns={'name_reg':'Регион','name_distr':'Район','name_county':'Округ/поселение','location':'Населенный пункт','population':'Численность населения','SiteID':'Ближайший сайт','distance':'Расстояние','SiteID_rdb':'Сайты на территории округа','Period':'Период','Subs':'Sim под покрытием'})
#7 Сохраняем
print('#7 Сохраняем')
ds.to_excel('./VOL/dic_VOL.xlsx',index=False)
ds

#1. Читаем данные из источников в формат GPD
#2 Формируем справочник по территориям
#2.1 Привязываем loc к county и distr (т.к. некоторые населенники принадлежат непосредственно району)
#2.2 Привязываем county к distr
#2.3. Привязываем distr_ID к reg_ID
#2.4 Объединяем
#3 Ищем ближайшую BS к каждой loc
#4 Привязываем BS к county и дополняем развернутой инфой
#6 Делаем красиво
#7 Сохраняем


,Регион,Район,Округ/поселение,Населенный пункт,Численность населения,Ближайший сайт,Расстояние,Период,Sim под покрытием,Сайты на территории округа
0,Вологодская область,Нюксенский район,Востровское сельское поселение,Стрелка,0,VG0154,0.144065,2020-04-01,90.0,VG0154
0,Вологодская область,Нюксенский район,Востровское сельское поселение,Стрелка,0,VG0154,0.144065,2020-05-01,90.0,VG0154
0,Вологодская область,Нюксенский район,Востровское сельское поселение,Стрелка,0,VG0154,0.144065,2020-06-01,106.0,VG0154
0,Вологодская область,Нюксенский район,Востровское сельское поселение,Стрелка,0,VG0154,0.144065,2020-07-01,115.0,VG0154
0,Вологодская область,Нюксенский район,Востровское сельское поселение,Стрелка,0,VG0154,0.144065,2020-08-01,111.0,VG0154
...,...,...,...,...,...,...,...,...,...,...
7797,Вологодская область,Верховажский район,Чушевицкое сельское поселение,Владыкина гора,95,VG0261,15.448636,2022-03-01,756.0,VG0261
7797,Вологодская область,Верховажский район,Чушевицкое сельское поселение,Владыкина гора,95,VG0261,15.448636,2022-04-01,747.0,VG0261
7797,Вологодская область,Верховажский район,Чушевицкое сельское поселение,Владыкина гора,95,VG0261,15.448636,2022-05-01,811.0,VG0261
7797,Вологодская область,Верховажский район,Чушевицкое сельское поселение,Владыкина гора,95,VG0261,15.448636,2022-06-01,895.0,VG0261


In [1]:
#3_BRY

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

#1. Читаем данные из источников в формат GPD
print('#1. Читаем данные из источников в формат GPD')
loc=pd.read_csv('population.csv')
loc=loc[loc['region']=='Брянская область'][['settlement','population','latitude_dd','longitude_dd']].reset_index(drop=True).rename(columns={'settlement':'location'})
loc['geometry']=loc.apply(lambda x:Point(x['longitude_dd'],x['latitude_dd']),axis=1)
loc=gpd.GeoDataFrame(loc[['location','population','geometry']]).rename(columns={'geometry':'geometry_loc'})

rdb=pd.read_excel('./RDB.xlsx')
rdb=rdb[rdb['MNEMO']=='BRY'][['SiteID','Lon','Lat']]
rdb['geometry']=rdb.apply(lambda x:Point(x['Lon'],x['Lat']),axis=1)
rdb=gpd.GeoDataFrame(rdb[['SiteID','geometry']])

ddb=pd.read_excel('./DDB.xlsx')
ddb=ddb[(ddb['MNEMO']=='BRY') & (ddb['PST_NAME']=='работает')][['POS_ID','Lon','Lat']]
ddb['geometry']=ddb.apply(lambda x:Point(x['Lon'],x['Lat']),axis=1)
ddb=gpd.GeoDataFrame(ddb[['POS_ID','geometry']])


#2 Формируем справочник по территориям
print('#2 Формируем справочник по территориям')
def readGEO(file,admin_level):
    geo=gpd.read_file(file)
    return geo[geo['admin_level']==str(admin_level)][['name','geometry']]

reg,distr,county=readGEO('./BRY/reg.geojson',4),readGEO('./BRY/distr.geojson',6),readGEO('./BRY/county.geojson',8)

#2.1 Привязываем loc к county и distr (т.к. некоторые населенники принадлежат непосредственно району)
print('#2.1 Привязываем loc к county и distr (т.к. некоторые населенники принадлежат непосредственно району)')
loc['county_ID']=loc.apply(lambda x:next(iter(county[county.contains(x['geometry_loc'])].index.values),None),axis=1)
loc['distr_ID']=loc.apply(lambda x:next(iter(distr[distr.contains(x['geometry_loc'])].index.values),None),axis=1)

#2.2 Привязываем county к distr
print('#2.2 Привязываем county к distr')
county['distr_ID']=county.apply(lambda x:next(iter(distr[distr.contains(x['geometry'])].index.values),None),axis=1)

#2.3. Привязываем distr_ID к reg_ID
print('#2.3. Привязываем distr_ID к reg_ID')
distr['reg_ID']=0

#2.4 Объединяем
print('#2.4 Объединяем')
ds=loc.join(county,on='county_ID',rsuffix='_county').join(distr,on='distr_ID',rsuffix='_distr').join(reg,on='reg_ID',rsuffix='_reg')
ds=ds.rename(columns={'name':'name_county','geometry':'geometry_county'})

#3 Ищем ближайшую BS к каждой loc
print('#3 Ищем ближайшую BS к каждой loc')
tmp=gpd.GeoDataFrame(ds[['geometry_loc']]).rename(columns={'geometry_loc':'geometry'}).sjoin_nearest(rdb,distance_col='distance').groupby(level=0).agg({'SiteID':lambda x:','.join(list(x)),'distance':['mean']})
tmp.columns=tmp.columns.droplevel(1)
tmp['distance']=tmp['distance']*111
ds=ds.join(tmp)

#4 Привязываем BS к county и дополняем развернутой инфой
print('#4 Привязываем BS к county и дополняем развернутой инфой')
rdb['county_ID']=rdb.apply(lambda x:next(iter(county[county.contains(x['geometry'])].index.values),None),axis=1)
rdb=rdb.join(pd.read_excel('./BSM.xlsx')[['SiteID','Period','Subs']].set_index('SiteID'),on='SiteID').dropna().groupby(['county_ID','Period'],as_index=False).agg({'Subs':['sum'],'SiteID':lambda x:','.join(list(x))}).set_index('county_ID')
rdb.columns=rdb.columns.droplevel(1)

#5 Привязываем POS к county и дополняем развернутой инфой
print('#5 Привязываем POS к county и дополняем развернутой инфой')
ddb['county_ID']=ddb.apply(lambda x:next(iter(county[county.contains(x['geometry'])].index.values),None),axis=1)

#5.1 Привязываем GI 3M
print('#5.1 Привязываем GI 3M')
gi=pd.read_excel('./BEL_Source.xlsx')[['Код точки продаж','GI','Дата']].rename(columns={'Код точки продаж':'POS_ID'})
gi=gi[gi['Дата']>'2022-05-01'].groupby(['POS_ID']).sum()

ddb=ddb.join(gi,on='POS_ID')
ddb=ddb.groupby(['county_ID'],as_index=False).agg({'POS_ID':lambda x:','.join(list(map(str,x))),'GI':'sum'}).set_index('county_ID')


#6 Делаем красиво
print('#6 Делаем красиво')
ds=ds[['name_reg','name_distr','name_county','location','population','SiteID','distance','county_ID']]
ds=ds.join(rdb,on='county_ID',rsuffix='_rdb').join(ddb,on='county_ID',rsuffix='_ddb')
ds=ds.drop(['county_ID'],axis=1)
ds=ds.rename(columns={'name_reg':'Регион','name_distr':'Район','name_county':'Округ/поселение','location':'Населенный пункт','population':'Численность населения','SiteID':'Ближайший сайт','distance':'Расстояние','SiteID_rdb':'Сайты на территории округа','Period':'Период','Subs':'Sim под покрытием','POS_ID':'ТТ на территории округа','GI':'GI 3M на территории округа'})
#7 Сохраняем
print('#7 Сохраняем')
ds.to_excel('dic_BRY.xlsx',index=False)
ds

C:\Users\mikhail.libman\AppData\Local\Continuum\anaconda3\lib\site-packages\geopandas\_compat.py:115: UserWarning: The Shapely GEOS version (3.7.0-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


#1. Читаем данные из источников в формат GPD
#2 Формируем справочник по территориям
#2.1 Привязываем loc к county и distr (т.к. некоторые населенники принадлежат непосредственно району)
#2.2 Привязываем county к distr
#2.3. Привязываем distr_ID к reg_ID
#2.4 Объединяем
#3 Ищем ближайшую BS к каждой loc
#4 Привязываем BS к county и дополняем развернутой инфой
#5 Привязываем POS к county и дополняем развернутой инфой
#5.1 Привязываем GI 3M
#6 Делаем красиво
#7 Сохраняем


,Регион,Район,Округ/поселение,Населенный пункт,Численность населения,Ближайший сайт,Расстояние,Период,Sim под покрытием,Сайты на территории округа,ТТ на территории округа,GI 3M на территории округа
0,Брянская область,Мглинский район,Ветлевское сельское поселение,Петровка,10,BR0171,8.375465,2020-04-01,217.0,BR0116,41045,0.0
0,Брянская область,Мглинский район,Ветлевское сельское поселение,Петровка,10,BR0171,8.375465,2020-05-01,213.0,BR0116,41045,0.0
0,Брянская область,Мглинский район,Ветлевское сельское поселение,Петровка,10,BR0171,8.375465,2020-06-01,200.0,BR0116,41045,0.0
0,Брянская область,Мглинский район,Ветлевское сельское поселение,Петровка,10,BR0171,8.375465,2020-07-01,206.0,BR0116,41045,0.0
0,Брянская область,Мглинский район,Ветлевское сельское поселение,Петровка,10,BR0171,8.375465,2020-08-01,201.0,BR0116,41045,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2705,Брянская область,Климовский район,Сачковичское сельское поселение,Дохновы,19,BR0736,1.553985,2022-04-01,773.0,BR0736,"78504,243057,243074,1000126",0.0
2705,Брянская область,Климовский район,Сачковичское сельское поселение,Дохновы,19,BR0736,1.553985,2022-05-01,606.0,BR0736,"78504,243057,243074,1000126",0.0
2705,Брянская область,Климовский район,Сачковичское сельское поселение,Дохновы,19,BR0736,1.553985,2022-06-01,646.0,BR0736,"78504,243057,243074,1000126",0.0
2705,Брянская область,Климовский район,Сачковичское сельское поселение,Дохновы,19,BR0736,1.553985,2022-07-01,624.0,BR0736,"78504,243057,243074,1000126",0.0
